# Mask R-CNN 

# 0. Импортируем библиотеки и загружаем данные

In [4]:
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from sklearn.model_selection import train_test_split
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader, DatasetCatalog, MetadataCatalog
import torch
import cv2

In [2]:
# Путь к изображениями
image_root = "E:/data/potato_leaf_dataset/images"

In [5]:
# Регистрируем COCO-датасет
register_coco_instances("potato_train", {}, "E:/data/potato_leaf_dataset/splits/instances_train.json", image_root)
register_coco_instances("potato_val", {}, "E:/data/potato_leaf_dataset/splits/instances_val.json", image_root)
register_coco_instances("potato_test", {}, "E:/data/potato_leaf_dataset/splits/instances_test.json", image_root)

namespace(name='potato_test',
          json_file='E:/data/potato_leaf_dataset/splits/instances_test.json',
          image_root='E:/data/potato_leaf_dataset/images',
          evaluator_type='coco',
          thing_classes=['leaf'])

# 1. Обучение модели

In [6]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
))

cfg.DATASETS.TRAIN = ("potato_train",)
cfg.DATASETS.TEST = ("potato_val",)
cfg.TEST.EVAL_PERIOD = 500
cfg.DATALOADER.NUM_WORKERS = 2  # больше воркеров

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)
cfg.SOLVER.IMS_PER_BATCH = 2  # больше батч
cfg.SOLVER.BASE_LR = 0.001  # lr под размер батча
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = [3000, 4500]  # lr decay для лучшего сходимости

cfg.SOLVER.AMP.ENABLED = True  # mixed precision

cfg.INPUT.MIN_SIZE_TRAIN = (800,)
cfg.INPUT.MAX_SIZE_TRAIN = 1333
cfg.INPUT.MAX_SIZE_TEST = 1333

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.OUTPUT_DIR = "./output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


In [16]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[06/16 16:24:19 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:474: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  grad_scaler = GradScaler()
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in

[06/16 16:24:19 d2.engine.train_loop]: Starting training from iteration 0


C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\torch\optim\lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precis

[06/16 16:24:35 d2.utils.events]:  eta: 0:52:42  iter: 19  total_loss: 3.5  loss_cls: 0.6901  loss_box_reg: 0.6814  loss_mask: 0.6904  loss_rpn_cls: 1.327  loss_rpn_loc: 0.104    time: 0.5579  last_time: 0.4845  data_time: 0.5584  last_data_time: 0.2698   lr: 4.9953e-06  max_mem: 2934M


...

[06/16 17:23:06 d2.utils.events]:  eta: 0:00:23  iter: 5959  total_loss: 0.283  loss_cls: 0.05271  loss_box_reg: 0.1001  loss_mask: 0.1055  loss_rpn_cls: 0.0003575  loss_rpn_loc: 0.02237    time: 0.5905  last_time: 0.3725  data_time: 0.3762  last_data_time: 0.1433   lr: 0.00025  max_mem: 2934M


C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dty

[06/16 17:23:19 d2.utils.events]:  eta: 0:00:11  iter: 5979  total_loss: 0.2552  loss_cls: 0.04127  loss_box_reg: 0.1006  loss_mask: 0.1003  loss_rpn_cls: 0.000566  loss_rpn_loc: 0.01658    time: 0.5906  last_time: 0.5715  data_time: 0.4124  last_data_time: 0.3486   lr: 0.00025  max_mem: 2934M


C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=self.precision):
C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\detectron2\engine\train_loop.py:493: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dty

[06/16 17:23:32 d2.utils.events]:  eta: 0:00:00  iter: 5999  total_loss: 0.3198  loss_cls: 0.04992  loss_box_reg: 0.1141  loss_mask: 0.1159  loss_rpn_cls: 0.0006073  loss_rpn_loc: 0.02425    time: 0.5907  last_time: 0.5644  data_time: 0.3618  last_data_time: 0.3061   lr: 0.00025  max_mem: 2934M
[06/16 17:23:32 d2.engine.hooks]: Overall training speed: 5998 iterations in 0:59:03 (0.5907 s / it)
[06/16 17:23:32 d2.engine.hooks]: Total training time: 0:59:06 (0:00:03 on hooks)
WARNING [06/16 17:23:32 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/16 17:23:32 d2.data.datasets.coco]: Loaded 32 images in COCO format from E:/data/potato_leaf_dataset/splits/instances_val.json
[06/16 17:23:32 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    leaf    | 352          |
|            |              |
[06/16 17:23:32 d2.data.dataset_mapper]: [DatasetM

# 2. Inference

In [2]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
))
cfg.DATASETS.TEST = ("potato_test",)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "mask_r_cnn.pth")  # путь к твоим весам
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # порог для визуализации

NameError: name 'get_cfg' is not defined

In [7]:
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultTrainer

# Создаем модель
model = DefaultTrainer.build_model(cfg)

# Загружаем веса
checkpointer = DetectionCheckpointer(model)
checkpointer.load(cfg.MODEL.WEIGHTS)

model.eval()

# Для тестового набора
test_loader = build_detection_test_loader(cfg, "potato_test")
test_evaluator = COCOEvaluator("potato_test", cfg, False, output_dir="./output/inference/")
print("Test set evaluation:")
print(inference_on_dataset(model, test_loader, test_evaluator))

# Для валидационного набора
val_loader = build_detection_test_loader(cfg, "potato_val")
val_evaluator = COCOEvaluator("potato_val", cfg, False, output_dir="./output/inference/")
print("Validation set evaluation:")
print(inference_on_dataset(model, val_loader, val_evaluator))



Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


Test set evaluation:


C:\Users\pavso\anaconda3\envs\plantcv\Lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.614
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.837
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.729
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.701
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.075
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.554
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.657
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.203
 Average Recall     (AR) @[ IoU=0.50:0.

In [9]:
predictor = DefaultPredictor(cfg)

In [12]:
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2
import matplotlib.pyplot as plt

image_path = "jpg/self.jpg"  # ← замени на свой путь
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 3. Инференс
outputs = predictor(image)

# 4. Визуализация
v = Visualizer(image_rgb, metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.0)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# 5. Отображение
plt.figure(figsize=(12, 8))
plt.imshow(out.get_image())
plt.axis("off")
plt.show()

AssertionError: Attribute 'thing_classes' in the metadata of 'coco_2017_train' cannot be set to a different value!
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'] != ['leaf']